# Heroes of Pymoli Data Analysis
## abstract analysis done on sample data: purchase_data.json
* Male players purchase more in-game items than any other genders.


* The age bracket of players that make the most purchases are:
    * 21 - 26 with about 36% of total in-game purchases
    * 16 - 21 with about 27% of total in-game purchases
    * 11 - 16 with about 12% of total in-game purchases


* One thing to note is that even though older age bracket players don't make as many purchases as younger age groups do, they do tend to spend on average more per item. They are willing to use their money on more costly items. The age group that is recognized as the top spender on a per-item basis are players of age 31-36 spending on average $3.08 per item.


* The items that sell the most tend to sell around the mid to lower $2 range.


* The more profitable items (assuming the cost of production for all items are equal) tend to sell around the mid to upper $4 range.

In [1]:
# import dependencies and mark options
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# file path variable
json_file = path.join('..', 'raw_data', 'purchase_data.json')

In [3]:
# read JSON file and preview it's contents
df = pd.read_json(json_file)

## Player Count

In [4]:
# total number of players
player_ct = len(df['SN'].unique())

# generate the wee-bit player count datafram
total_player_ct_df = pd.DataFrame({'Total Players': player_ct}, index=['HOP Summary Value'])
total_player_ct_df

,Total Players
HOP Summary Value,573


## Unique Player DataFrame Generator
In order to get accurate summaries on any data that doesn't explicitly include purchasing items than it is imperative to use a unique dataframe because some users are duplicated and for that reason some data may be redundant.

In [5]:
# unique player gender extractor
unique_players = df['SN'].unique()

unique_plyr_age = [list(df['Age'].loc[df['SN'] == str(i)]) for i in unique_players]
unique_plyr_age = [item[0] for item in unique_plyr_age]

unique_gender = [list(df['Gender'].loc[df['SN'] == str(i)]) for i in unique_players]
unique_gender = [item[0] for item in unique_gender]

# create a dataframe out of unique_index list
unique_player_df = pd.DataFrame({
    'uPlayer SN': unique_players,
    'uPlayer Gender': unique_gender,
    'uPlayer Age': unique_plyr_age
})

## Purchasing Analysis (Total)

In [6]:
# purchase analysis
itemID_unique_ct = len(df['Item ID'].unique())
itemName_unique_ct = len(df['Item Name'].unique())

avg_item_cost = df['Price'].mean()
avg_item_cost = "${:.2f}".format(avg_item_cost)

num_purchases = len(df)

total_rev = df['Price'].sum()
total_rev = "${:.2f}".format(total_rev)

# generate purchasing analysis datafram
purchase_analysis_df = pd.DataFrame({
    'Number of Unique Items': itemName_unique_ct,
    'Average Price': avg_item_cost,
    'Number of Purchases': num_purchases,
    'Total Revenue': total_rev
}, index=['HOP Summary Value'], columns=[
    'Number of Unique Items',
    'Average Price',
    'Number of Purchases',
    'Total Revenue'
])
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
HOP Summary Value,179,$2.93,780,$2286.33


## Gender Demographics

In [7]:
# gender demographics
male_ct = len(unique_player_df.loc[unique_player_df['uPlayer Gender'] == 'Male'])
female_ct = len(unique_player_df.loc[unique_player_df['uPlayer Gender'] == 'Female'])
other_gender_ct = len(unique_player_df.loc[unique_player_df['uPlayer Gender'] == 'Other / Non-Disclosed'])

male_perc = "{:.2f}%".format((male_ct/player_ct)*100)
female_perc = "{:.2f}%".format((female_ct/player_ct)*100)
other_perc = "{:.2f}%".format((other_gender_ct/player_ct)*100)

# generate gender anaylsis dataframe
gender_analysis_df = pd.DataFrame({
    'Percentage of Players': [male_perc, female_perc, other_perc],
    'Total Count': [male_ct, female_ct, other_gender_ct]
}, index=['Male', 'Female', 'Other / Non-Disclosed'])

gender_analysis_df

,Percentage of Players,Total Count
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclosed,1.40%,8


## Purchasing Analysis (Gender)
I am pretty sure the normalized total values are incorrect

In [8]:
# purchasing gender count
p_male_ct = len(df.loc[df['Gender']=='Male'])
p_female_ct = len(df.loc[df['Gender']=='Female'])
p_other_gender_ct = len(df.loc[df['Gender']=='Other / Non-Disclosed'])

# avg gender purchase price
gender_avg_groupby_df = df.groupby(by='Gender').mean()
male_avg_pp = gender_avg_groupby_df['Price']['Male']
female_avg_pp = gender_avg_groupby_df['Price']['Female']
other_gender_avg_pp = gender_avg_groupby_df['Price']['Other / Non-Disclosed']

# total purchase value
gender_sum_groupby_df = df.groupby(by='Gender').sum()
male_sum_pp = gender_sum_groupby_df['Price']['Male']
female_sum_pp = gender_sum_groupby_df['Price']['Female']
other_gender_sum_pp = gender_sum_groupby_df['Price']['Other / Non-Disclosed']

# normalized totals
male_norm_totals = male_avg_pp * p_male_ct
female_norm_totals = female_avg_pp * p_female_ct
other_gender_norm_totals = other_gender_avg_pp * p_other_gender_ct

# format figures
format_list = [
    male_avg_pp, female_avg_pp, other_gender_avg_pp,
    male_sum_pp, female_sum_pp, other_gender_sum_pp,
    male_norm_totals, female_norm_totals, other_gender_norm_totals
]

formatted = ["${:.2f}".format(item) for item in format_list]


# generate the purchasing gender analysis dataframe
p_gender_analaysis_df = pd.DataFrame({
    'Purchase Count': [p_male_ct, p_female_ct, p_other_gender_ct],
    'Average Purchase Price': [formatted[0], formatted[1], formatted[2]],
    'Total Purchase Value': [formatted[3], formatted[4], formatted[5]],
    'Normalized Totals': [formatted[6], formatted[7], formatted[8]]
}, index=['Male','Female', 'Other / Non-Disclosed'], columns=[
    'Purchase Count',
    'Average Purchase Price',
    'Total Purchase Value',
    'Normalized Totals'
])
p_gender_analaysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Male,633,$2.95,$1867.68,$1867.68
Female,136,$2.82,$382.91,$382.91
Other / Non-Disclosed,11,$3.25,$35.74,$35.74


### bins and labels for age feature

In [9]:
# locate the max and min for player ages
u_min = unique_player_df['uPlayer Age'].min()
u_max = unique_player_df['uPlayer Age'].max()

# dynamic bin generator
bins = list(np.linspace(u_min - 1,u_max + 1,num=(u_max-u_min)/4, dtype=int))

# dynamic label generator
labels = ["<{}".format(bins[1])]
for k in range(len(bins)):
    labels.append("{0}-{1}".format(bins[k-1], bins[k]))
del labels[1:3]
del labels[-1]
labels.append("{}+".format(bins[-2]))

## Age Demographics

In [10]:
# bin cut made to the unique_player_df 
unique_player_df['uPlayer Age Range'] = pd.cut(list(unique_player_df['uPlayer Age']), bins=bins, labels=labels)

# generate a new dataframe using the groupby method on players' ages
age_range_groupby_df = unique_player_df.groupby(by='uPlayer Age Range').count()

age_range_groupby_df = age_range_groupby_df[['uPlayer Age', 'uPlayer Gender']]
age_range_groupby_df = age_range_groupby_df.rename(columns={
    'uPlayer Age': 'Percentage of Players',
    'uPlayer Gender': 'Total Count'
})
age_perc = (age_range_groupby_df['Percentage of Players'] / player_ct)*100
age_range_groupby_df['Percentage of Players'] = age_perc.map("{:.2f}%".format)
del age_range_groupby_df.index.name
age_range_groupby_df

,Percentage of Players,Total Count
<11,4.71%,27
11-16,11.87%,68
16-21,26.88%,154
21-26,36.30%,208
26-31,9.42%,54
31-36,6.46%,37
36-41,3.84%,22
41+,0.52%,3


## Purchasing Analysis (Age)
I am pretty sure the normalized totals values are incorrect

In [11]:
df['Player Age Range'] = pd.cut(df['Age'], bins=bins, labels=labels)
p_age_range_groupby_ct_df = df.groupby(by='Player Age Range').count()
p_age_range_groupby_mn_df = df.groupby(by='Player Age Range').mean()
p_age_range_groupby_sm_df = df.groupby(by='Player Age Range').sum()
p_age_range_groupby_std_df = df.groupby(by='Player Age Range').std()

normalized = (p_age_range_groupby_sm_df['Price'] - p_age_range_groupby_mn_df['Price'])/p_age_range_groupby_std_df['Price']

# filter rows
p_age_range_groupby_mn_df = p_age_range_groupby_mn_df['Price'].map("${:.2f}".format)
p_age_range_groupby_sm_df = p_age_range_groupby_sm_df['Price'].map("${:.2f}".format)
normalized = normalized.map("${:.2f}".format)


p_age_range_groupby_ct_df = p_age_range_groupby_ct_df[['Age']]
p_age_range_groupby_ct_df = p_age_range_groupby_ct_df.rename(columns={'Age': 'Purchase Count'})
p_age_range_groupby_ct_df['Average Purchase Price'] = p_age_range_groupby_mn_df
p_age_range_groupby_ct_df['Total Purchase Value'] = p_age_range_groupby_sm_df
p_age_range_groupby_ct_df['Normalized Totals'] = normalized



del p_age_range_groupby_ct_df.index.name
p_age_range_groupby_ct_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
<11,41,$3.01,$123.38,$108.71
11-16,92,$2.81,$258.10,$231.31
16-21,204,$2.88,$588.40,$525.26
21-26,275,$2.96,$814.07,$714.52
26-31,79,$2.98,$235.61,$200.83
31-36,49,$3.08,$150.78,$146.21
36-41,37,$2.90,$107.35,$92.61
41+,3,$2.88,$8.64,$6.69


## Top Spenders

In [12]:
top_spender_sn_df = df.groupby(by='SN').sum().sort_values(['Price'], ascending=False)[['Price']]

top_spender_ct_df = df.groupby(by='SN').count()
top_spender_mn_df = df.groupby(by='SN').mean()


# list comprehensions for additional columns
tp_index = list(top_spender_sn_df.index)
tp_count = [top_spender_ct_df['Price'][element] for element in tp_index]
tp_mean = [top_spender_mn_df['Price'][element] for element in tp_index]

# top_spender dataframe build
top_spender_sn_df['Purchase Count'] = tp_count
top_spender_sn_df['Average Purchase Price'] = tp_mean

top_spender_sn_df = top_spender_sn_df.rename(columns={'Price':'Total Purchase Value'})
top_spender_sn_df = top_spender_sn_df[[
    'Purchase Count',
    'Average Purchase Price',
    'Total Purchase Value'
]]

# format dataframe
top_spender_sn_df['Average Purchase Price'] = top_spender_sn_df['Average Purchase Price'].map("${:.2f}".format)
top_spender_sn_df['Total Purchase Value'] = top_spender_sn_df['Total Purchase Value'].map("${:.2f}".format)

top_spender_sn_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


## Most Popular Items

In [13]:
popular_items_id_df = df.groupby(by=['Item ID','Item Name']).count().sort_values(['Price'], ascending=False)

popular_items_sm_df = df.groupby(by=['Item ID','Item Name']).sum()
popular_items_mn_df = df.groupby(by=['Item ID','Item Name']).mean()

# list comprehensions for additional columns
pop_index = list(popular_items_id_df.index)
pop_sum = [popular_items_sm_df['Price'][element] for element in pop_index]
pop_mean = [popular_items_mn_df['Price'][element] for element in pop_index]

# popular_items dataframe build
popular_items_id_df['Item Price'] = pop_mean
popular_items_id_df['Total Purchase Value'] = pop_sum

popular_items_id_df = popular_items_id_df.rename(columns={'Price':'Purchase Count'})
popular_items_id_df = popular_items_id_df[[
    'Purchase Count',
    'Item Price',
    'Total Purchase Value'
]]

# format dataframe
popular_items_id_df['Item Price'] = popular_items_id_df['Item Price'].map("${:.2f}".format)
popular_items_id_df['Total Purchase Value'] = popular_items_id_df['Total Purchase Value'].map("${:.2f}".format)

popular_items_id_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


## Most Profitable Items

In [14]:
profitable_items_id_df = df.groupby(by=['Item ID','Item Name']).sum().sort_values(['Price'], ascending=False)

profitable_items_ct_df = df.groupby(by=['Item ID','Item Name']).count()
profitable_items_mn_df = df.groupby(by=['Item ID','Item Name']).mean()

# list comprehensions for additional columns
prof_index = list(profitable_items_id_df.index)
prof_count = [profitable_items_ct_df['Price'][element] for element in prof_index]
prof_mean = [profitable_items_mn_df['Price'][element] for element in prof_index]

# popular_items dataframe build
profitable_items_id_df['Purchase Count'] = prof_count
profitable_items_id_df['Item Price'] = prof_mean

profitable_items_id_df = profitable_items_id_df.rename(columns={'Price':'Total Purchase Value'})
profitable_items_id_df = profitable_items_id_df[[
    'Purchase Count',
    'Item Price',
    'Total Purchase Value'
]]

# format dataframe
profitable_items_id_df['Item Price'] = profitable_items_id_df['Item Price'].map("${:.2f}".format)
profitable_items_id_df['Total Purchase Value'] = profitable_items_id_df['Total Purchase Value'].map("${:.2f}".format)

profitable_items_id_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
